In [1]:
%run '00_rh_settings.ipynb'

Populating the interactive namespace from numpy and matplotlib


# Correlation and non-independence

In [2]:
#data_GA = data.query("Stock == 'GOOG' | Stock == 'AAPL'").dropna().groupby("Stock")["Return"]
data_AFG = data.pivot_table(index='Date', columns='Stock', values='Return')[["AAPL","FB","GOOG"]].dropna()
data_AFG.head()

Stock,AAPL,FB,GOOG
Date,,,
2014-03-28,0.001118,0.015997,-0.002732
2014-03-31,0.000224,-0.003818,0.005422
2014-04-01,-0.009065,-0.038007,-0.017967
2014-04-02,-0.001659,-0.001594,0.000282
2014-04-03,0.006979,0.054295,-0.004809


In [3]:
#data.query("Stock == 'GOOG' | Stock == 'AAPL' | Stock == 'FB'")[["Date","Stock","Return"]].dropna().groupby(["Date","Stock"], as_index=False).head()

## Granger causality

In [4]:
statsmodels.api.tsa.stattools.grangercausalitytests(data_AFG[['GOOG', 'AAPL']], maxlag=10, addconst=True);

AttributeError: module 'statsmodels' has no attribute 'api'

__Sum up:__ Apple and Google stocks has no common lag with significant value of linear interdependence

## Using correlation is a bad idea, so we use EGCM package recently ported from R-package

In [ ]:
stats.pearsonr(data_AFG.AAPL, data_AFG.GOOG)

To python package converted only a subset of functions, the package is raw, but it is still better than Pearson correlation.

Link to original R package: https://cran.r-project.org/web/packages/egcm/index.html

Python port link: https://github.com/David-OConnor/egcm

In [ ]:
from egcm.test import *
from egcm import *

Citation from documentation (https://github.com/cran/egcm/blob/master/R/egcm.R):

    Given two time series $Y[t]$ and $X[t]$, the Engle Granger cointegration
    model in its simplest form looks for alpha, beta and rho such that

    $Y[t] = \alpha + \beta * X[t] + R[t]$

    $R[t] = \rho * R[t-1] + \epsilon[t]$

    where $epsilon[t]$ is a series of independent and identically distributed
    innovations with mean zero. If $\alpha$, $\beta$ and $\rho$ can be found such that
    $-1 < \rho < 1$, then the series are said to be cointegrated. If $abs(\rho) = 1$,
    then the residual series $R[t]$ is said to have a unit root (or alternatively,
    it is said to follow a random walk).

Looks like, ported package wrong calculate P-value, so we implement additional check.

In [ ]:
def cointegration_check(egcm_result):
    # Check for cointegration without pvalue
    keys = ['alpha', 'beta','rho']
    parameters = [egcm_result.get(key) for key in keys]
    df = pd.DataFrame({'keys': keys, 'parameters': parameters})
    return df, all(abs(df.parameters) < 1)

### Test data

In [ ]:
norm_values = scipy.stats.norm.rvs(size=100)

test_data0 = pd.DataFrame({"First":norm_values, "Second":200*norm_values+200})
test_data1 = pd.DataFrame({"First":norm_values, "Second":scipy.stats.norm.rvs(size=100)})
test_data2 = pd.DataFrame({"First":norm_values, "Second":norm_values+norm_values})
test_data3 = pd.DataFrame({"First":norm_values, "Second":range(0,100)+sin(range(0,100))+norm_values+norm_values*norm_values})
test_data4 = pd.DataFrame({"First":norm_values, "Second":range(0,100)})
test_data = test_data1

test_egcm_result = egcm(test_data["First"], test_data["Second"], robust=True)
test_egcm(test_egcm_result)
is_cointegrated(test_egcm_result)
cointegration_check(test_egcm_result)

## Apple and Google

In [ ]:
#import egcm
egcm_result = egcm(X=data_AFG.AAPL, Y=data_AFG.GOOG, robust=True)

# egcm(X,Y) -- Constructs an Engle-Granger cointegration model from X & Y
# summary.egcm(E) -- Prints various summary statistics on the Engle-Granger
# cointegration model constructed from X & Y
# plot.egcm(E) -- Creates a graph of the Engle-Granger cointegration model
test_egcm(egcm_result)
is_cointegrated(egcm_result)
cointegration_check(egcm_result)

We cannot reject that they co-integrated